In [1]:
import os
import sys
from src import data_prepare

import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [2]:
post, thread=data_prepare.load_train_data()
label_map=data_prepare.load_label_map()
thread.head()

In [3]:
key_words=pd.Series(label_map.index)
key_words

0             bastard
1     beginners-mafia
2                byor
3             classic
4        closed-setup
5              cybrid
6                kotm
7      non-mafia-game
8               other
9          paranormal
10       supernatural
11            vanilla
12           vengeful
Name: type_name, dtype: object

In [4]:
key_words[1]="beginner"
key_words[6]="king"
thread["name"]=thread["thread_name"].apply(lambda x: data_prepare.clean(x)).as_matrix()

In [75]:
correct=pd.Series(np.zeros(13),index=label_map.index)
over=pd.Series(np.zeros(13),index=label_map.index)
under=pd.Series(np.zeros(13),index=label_map.index)
al=pd.Series(np.zeros(13),index=label_map.index)

In [78]:
for item in thread.itertuples():
    label=item.thread_label_id
    if key_words[label] in item.name:
        correct[label]+=1
    else:
        under[label]+=1
    for index,s in enumerate(key_words):
        if index==label:
            continue
        if s in item.name:
            over[index]+=1
            break
    al[label]+=1

In [81]:
x = pd.DataFrame({'all':al,'correct':correct,'under': under,'over':over,
                  'per cent correct':correct/al,'per cent under':under/al,
                 'per cent over':over/al},
                 index=correct.index)
x

,all,correct,under,over,per cent correct,per cent under,per cent over
type_name,,,,,,,
bastard,28.0,14.0,14.0,4.0,0.500000,0.500000,0.142857
beginners-mafia,46.0,42.0,4.0,13.0,0.913043,0.086957,0.282609
byor,26.0,16.0,10.0,4.0,0.615385,0.384615,0.153846
classic,42.0,0.0,42.0,1.0,0.000000,1.000000,0.023810
closed-setup,72.0,0.0,72.0,0.0,0.000000,1.000000,0.000000
cybrid,6.0,6.0,0.0,1.0,1.000000,0.000000,0.166667
kotm,4.0,4.0,0.0,16.0,1.000000,0.000000,4.000000
non-mafia-game,4.0,0.0,4.0,0.0,0.000000,1.000000,0.000000
other,402.0,12.0,390.0,1.0,0.029851,0.970149,0.002488


In [82]:
post_test, thread_test=data_prepare.load_test_data()

In [84]:
a=pd.Series(np.zeros(13),index=label_map.index)
thread_test["name"]=thread_test["thread_name"].apply(lambda x: data_prepare.clean(x)).as_matrix()
for item in thread_test.itertuples():
    for index,s in enumerate(key_words):
        if s in item.name:
            a[index]+=1

In [85]:
a

type_name
bastard             8.0
beginners-mafia    35.0
byor               17.0
classic             0.0
closed-setup        0.0
cybrid              5.0
kotm                8.0
non-mafia-game      0.0
other               0.0
paranormal          7.0
supernatural        7.0
vanilla             2.0
vengeful            6.0
dtype: float64